In [ ]:
# !sed -i 's/transformers.generation_utils/transformers.generation/' /usr/local/lib/python3.12/dist-packages/openprompt/pipeline_base.py

In [ ]:
# !pip install transformers==4.24.0

In [1]:
import os
import shutil

# Get user profile path
user_profile = os.environ["USERPROFILE"]

# Paths to Hugging Face cached models
cached_models = [
    os.path.join(user_profile, r".cache\huggingface\hub\models--bert-base-multilingual-cased"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--xlm-roberta-base"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--roberta-base")
]

# Remove cached models if they exist
for path in cached_models:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Removed cache: {path}")
    else:
        print(f"No cache found at: {path}")


Removed cache: C:\Users\stdFurqan\.cache\huggingface\hub\models--bert-base-multilingual-cased
No cache found at: C:\Users\stdFurqan\.cache\huggingface\hub\models--xlm-roberta-base
No cache found at: C:\Users\stdFurqan\.cache\huggingface\hub\models--roberta-base


In [2]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
import torch

print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [4]:
# ==============================
# Imports
# ==============================
import pandas as pd
import torch
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.prompts import SoftVerbalizer
from openprompt.prompts import AutomaticVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:


# ==============================
# Set random seeds for reproducibility
# ==============================
# 💡 Added this block to ensure consistent results across runs
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:

# Step 1: Training data (16-shot) ha,m and spam
train_dataset = [

    InputExample(guid=0, text_a="کراچی: باپ بیٹے اورسابق ایف آئی اے ملازم سمیت 5 افراد ہلاک", label=0),
    InputExample(guid=1, text_a="دلالوں کی اصلیت اب کسی سے چھپی نہیں رہی ہے ۔ کل تک جو نظام لپیٹنے کی باتیں کر رہے تھے آج کھسیانے ہو کر اسی نظام۔۔۔", label=0),
    InputExample(guid=2, text_a="فلسفہِ محبت کی تقریب ِرونمائی براہ راست نشر کیوں نہیں کی؟", label=0),
    InputExample(guid=3, text_a="تم سے بڑا گھپلا کون ہے", label=0),
    InputExample(guid=4, text_a="قادری انکل نے جو کفن سِلوایا تھا نواز حکومت کے لیے وہ کہاں گیا؟", label=0),
    InputExample(guid=5, text_a="لاہور:پسند کی شادی کرنے والی لڑکیاں پہلے حالات کیوں نہیں سمجھتیں ۔ جسٹس قاسم خان", label=0),
    InputExample(guid=6, text_a="چھاپہ مار ٹیم نے ٹاپ ٹریٹمنٹ کے نام سے قائم فیکٹری کو سیل کردیا", label=0),
    InputExample(guid=7, text_a="ﺳﻮﺩﮐﮭﺎﻧﮯ ﻭﺍﻟﮯ، ﺍﺱ ﮐﮯ ﻭﮐﯿﻞ، ﺍﺳﮑﮯ ﮐﺎﺗﺐ ﺍﻭﺭ ﺍﺱ ﮐﮯ ﮔﻮﺍﮦ ﭘﺮ ﻟﻌﻨﺖ ﻓﺮﻣﺎﺋﯽ ﮨﮯ ﺍﻭﺭ ﻓﺮﻣﺎﯾﺎ ﮐﮧ ﻭﮦ ﺳﺎﺭﮮ ﺑﺮﺍﺑﺮ ﮨﯿﮟفرمان S A W W", label=0),
    InputExample(guid=8, text_a="مسجد میں ریپ کرنے والے امام کو 20 سال قید۔۔صرف بیس مزید پڑھیں", label=0),
    InputExample(guid=9, text_a="کمال کی داستان ہے سنو میں نے جس کے لیے سب کو چھوڑا دیا اس نے مجھے چھوڑ دیا", label=0),
    InputExample(guid=10, text_a="ہاں تم ویسے ہی پراسرار ہے اور پراسرار ڈائنیں ہوتی ہیں", label=0),
    InputExample(guid=11, text_a="اپنے باپ شوبازشریف کو بول پہلے وہ پنجاب کو بچالے اور تو اپنی بجلی پر توجہ دے ۔ ماما نا بن خیبر پختونخواہ کا۔جعلی معزز بهڑوے سالے", label=0),
    InputExample(guid=12, text_a="سابق چیف جسٹس پاکستان کو شدید خطرات لاحق via", label=0),
    InputExample(guid=13, text_a="منافق کی تین نشانیاں ہیں جب بات کرے توجھوٹ بولے ، جب وعدہ کرے تواس کی خلاف ورزی کرے اور جب امانت رکھوائی جائے تواس میں خیانت کرے ۔(بخاری)", label=0),
    InputExample(guid=14, text_a="بلدیاتی انتخابات میں التوا کا ذمہ دار الیکشن کمیشن ہے :عمران خان", label=0),
    InputExample(guid=15, text_a="تحفظات دور کرنے کی بجائے معطل کر دیا گیا : جاوید نسیم", label=0),

    
    InputExample(guid=16, text_a="پوتن نے روس اور اسرائیل کے دوستانہ تعلقات میں نیتن یاہو کے بڑے کردار کو بے حد سراہا", label=1),
    InputExample(guid=17, text_a="کوئی بھی شخص اس غلط فہمی میں نہ رہے", label=1),
    InputExample(guid=18, text_a="او جی ڈی سی ایل شیئرز اور سکوک :غیر ملکی ادائیگیوں میں بہتری کی امید: اسلام آباد۔۔۔۔۔۔۔۔۔آئن۔۔۔", label=1),
    InputExample(guid=19, text_a="خوبصورتی کی انتہاء ہو گئی ۔۔۔۔", label=1),
    InputExample(guid=20, text_a="سبزے کو جب کہیں جگہ نہ ملی بن گیا روئے آب پر کائی ۔۔ غالب", label=1),
    InputExample(guid=21, text_a="حدیثا الله کاذکر ایمان کاجھنڈا شیطان سے حفاظت کا قلعه اور بھڑکتی آگ (نارجهنم) سے نجات کاذریعه ھے )گلدستهء فقر صفحه118(", label=1),
    InputExample(guid=22, text_a="هم اپنے حصے کا پاکستان ساتھ لائے ، پهلے پاکستان بنایا پهر یهاں آئے ، خواجه اظهار ایم کیو ایم", label=1),
    InputExample(guid=23, text_a="تہمتیں تو لگتی ہیں روشنی کی خواہش میں گھر سے باہر آنے کی کُچھ سزا تو ملتی ہے لوگ لوگ ہوتے ان کو کیا خبر جاناں !۔۔۔", label=1),
    InputExample(guid=24, text_a="عمران خان کنٹینرسے نکل کراس کی چھت پر پہنچے اورکئی لمحے طاہر القادری کے کنٹینرکی طرف لوگوں کوسامان باندھتا دیکھتے رہے ۔", label=1),
    InputExample(guid=25, text_a="پیار بھرے دو شرمیلے نین چندے سے انقلاب", label=1),
    InputExample(guid=26, text_a="مُجھے کر عطا اے میرے اللہ، توُ بہت بندہ نواز ہے ۔ ۔ ۔میری ہر صبح محتاج ہے تیری رحمتوں کے نزول کی ۔ ۔ ۔دعاء پینڈو", label=1),
    InputExample(guid=27, text_a="بنوں میں دوستانہ میچ آئی ڈی پیز کی شاہد آفریدی الیون نے جیت لیا", label=1),
    InputExample(guid=28, text_a="ابھی تو پارٹی شروع ہوئی ہے", label=1),
    InputExample(guid=29, text_a="حضرت امام حسین کا صبر شکر توکل اور رضائے الٰہی کائنات کے انسانوں کیلئے علم و عرفان کا مینار ہیں ‘ ۔۔۔", label=1),
    InputExample(guid=30, text_a="ر ایک بچے کی سکول میں موجودگی یقینی بنانے کے لئے ریاست کو تعلیمی معاملات سے نمٹنے کے لئے نئی قوت کے ساتھ عزم کو یقینی…", label=1),
    InputExample(guid=31, text_a="جمعه مبارک ، دعا میں یاد رکھیں ،شکریه", label=1),    
]


In [7]:
# ==============================
# Define Classes
# ==============================
classes = ["neg", "pos"]
label_map = {"neg": 0, "pos": 1}

# # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
_, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # Step 2: Manually load XLM-RoBERTa model/tokenizer
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
# plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\stdFurqan\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an 

In [8]:

# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
    # text = '{"placeholder":"text_a"} هذه الجملة {"mask"}.',
    text='{"placeholder":"text_a"} یہ جملہ {"mask"} ہے۔',
    tokenizer=tokenizer,
)
 
# ==============================
# Define Verbalizer (Manual)
# ==============================
verbalizer = ManualVerbalizer(
    classes=classes,
    label_words = {
    "neg": ["منفی", "غلط"],   # Negative / undesirable
    "pos": ["مثبت", "صحیح"]
    },
    tokenizer=tokenizer,
)

In [9]:



# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)

# ==============================
# DataLoader for Training
# ==============================
train_loader = PromptDataLoader(
    dataset=train_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=4,
    shuffle=True  # Even though shuffled, reproducibility is preserved by seeding
)

# # ==============================
# # Fine-Tuning the Prompt Model
# # ==============================
prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)

for epoch in range(20):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")


 

# ==============================
# Load Evaluation Dataset
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\SST-2\urdu_sentiment_test_labeled.csv")
eval_dataset = [
    InputExample(guid=i, text_a=row['text'], label=label_map[row['label']])
    for i, row in df.iterrows()
]


eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 Fine-tuned Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))


tokenizing: 32it [00:00, 3033.85it/s]


Epoch 1 Loss: 5.7101
Epoch 2 Loss: 5.4283
Epoch 3 Loss: 3.5730
Epoch 4 Loss: 1.1229
Epoch 5 Loss: 0.8077
Epoch 6 Loss: 0.4491
Epoch 7 Loss: 0.0570
Epoch 8 Loss: 0.0408
Epoch 9 Loss: 0.0451
Epoch 10 Loss: 0.0225
Epoch 11 Loss: 0.0203
Epoch 12 Loss: 0.0186
Epoch 13 Loss: 0.0265
Epoch 14 Loss: 0.0113
Epoch 15 Loss: 0.0127
Epoch 16 Loss: 0.1779
Epoch 17 Loss: 0.0206
Epoch 18 Loss: 0.0273
Epoch 19 Loss: 0.0177
Epoch 20 Loss: 0.0068


tokenizing: 294it [00:00, 3266.68it/s]



📊 Fine-tuned Classification Report:
              precision    recall  f1-score   support

         neg     0.7622    0.8278    0.7937       151
         pos     0.8000    0.7273    0.7619       143

    accuracy                         0.7789       294
   macro avg     0.7811    0.7775    0.7778       294
weighted avg     0.7806    0.7789    0.7782       294

